In [1]:
import pandas as pd


train_data = pd.read_csv("data\main\df_train_b1.csv", index_col=0)
test_data = pd.read_csv('data\main\df_test_b1.csv', index_col=0)

In [2]:
if True:

    import pandas as pd
    temp_df = pd.read_csv("./data/detail/df_loan1_b1.csv")

    def chuli_loan(df, colname, ways):
        cust_id = list(range(len(set(df['cust_id']))))
        res_df = pd.DataFrame({'cust_id': cust_id})

        if 'mean' in ways:
            res_df[colname+'_mean'] = df.groupby('cust_id')[colname].mean()
        if 'sum' in ways:
            res_df[colname+'_sum'] = df.groupby('cust_id')[colname].sum()
        if 'count' in ways:
            res_df[colname+'_count'] = df.groupby('cust_id')[colname].count()
        if 'std' in ways:
            res_df[colname+'_std'] = df.groupby('cust_id')[colname].std()
        if 'min' in ways:
            res_df[colname+'_min'] = df.groupby('cust_id')[colname].min()
        if 'max' in ways:
            res_df[colname+'_max'] = df.groupby('cust_id')[colname].max()
        if 'diff_std' in ways:
            res_df[colname+'_diff_std'] = df.groupby('cust_id')[colname].apply(
                lambda x: x.sort_values(ascending=True).diff()[2:].std())
        return res_df

    for colname in ['loan1_2', 'loan1_3', 'loan1_7', 'loan1_9']:
        df = chuli_loan(temp_df, colname, ('sum',))
        train_data = pd.merge(train_data, df, on="cust_id",
                              how="left", suffixes=("", "_y"))
        test_data = pd.merge(test_data, df, on="cust_id",   
                             how="left", suffixes=("", "_y"))
    # for colname in ['loan1_2','loan1_3']:
    #     df = chuli_loan(temp_df,colname , ('count',))
    #     train_data = pd.merge(train_data, df, on="cust_id",
    #                             how="left", suffixes=("", "_y"))
    #     test_data = pd.merge(test_data, df, on="cust_id",
    #                             how="left", suffixes=("", "_y"))                    

In [3]:
df_train=train_data
df_test=test_data

In [4]:
df_train.drop(["loan1_17",'loan1_31'],axis=1,inplace=True)
df_test.drop(["loan1_17",'loan1_31'],axis=1,inplace=True)

In [4]:
def delnan(df):
    df = df.dropna(how="all", axis=1)  # 删除全是空值的列
    df = df.fillna(-99)
    return df

df_train = delnan(train_data)
df_test = delnan(test_data)

In [5]:
full_col_name = list(df_train.columns)
basic_col = [i for i in full_col_name if "basic" in i]
loan_col = [i for i in full_col_name if "loan" in i]
overdue_col = [i for i in full_col_name if "overdue" in i]
query_col = [i for i in full_col_name if "query" in i]
judicial_col = [i for i in full_col_name if "judicial" in i]
date_col = [i for i in full_col_name if "date" in i]

In [6]:
x_col = basic_col+['is_judicial']+loan_col+query_col
x_col=[i for i in x_col if i in full_col_name]

In [7]:
print(x_col)

['basic_1', 'basic_2', 'basic_3', 'basic_4', 'basic_5', 'basic_6', 'basic_7', 'basic_8', 'basic_9', 'basic_10', 'basic_11', 'basic_12', 'basic_13', 'basic_14', 'basic_15', 'loan1_1', 'loan1_2', 'loan1_3', 'loan1_4', 'loan1_5', 'loan1_6', 'loan1_7', 'loan1_8', 'loan1_9', 'loan1_10', 'loan1_11', 'loan1_12', 'loan1_13', 'loan1_14', 'loan1_15', 'loan1_16', 'loan1_17', 'loan1_18', 'loan1_19', 'loan1_20', 'loan1_21', 'loan1_22', 'loan1_23', 'loan1_24', 'loan1_25', 'loan1_26', 'loan1_27', 'loan1_28', 'loan1_29', 'loan1_30', 'loan1_31', 'loan1_32', 'loan1_33', 'loan2_1', 'loan2_2', 'loan2_3', 'loan2_4', 'loan2_5', 'loan2_6', 'loan2_7', 'loan2_8', 'loan2_9', 'loan2_10', 'loan2_11', 'loan2_12', 'loan1_2_sum', 'loan1_3_sum', 'loan1_7_sum', 'loan1_9_sum', 'query_1', 'query_2', 'query_3', 'query_4', 'query_5', 'query_6', 'query_7', 'query_8', 'query_9', 'query_10', 'query_11']


In [8]:
import copy
import xgboost as xgb
import random
from sklearn import model_selection
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
from lightgbm import early_stopping, log_evaluation


In [9]:
X = df_train[x_col]
Y = df_train['label']

X_train, X_test, y_train, y_test = model_selection.train_test_split(
    X, Y, test_size=3000, stratify=Y, random_state=2022)

In [11]:
dtrain=xgb.DMatrix(X,label=Y)

param = {'max_depth': 5,
         'eta': 0.008,
         'objective': 'binary:logistic',
         "eval_metric":"auc",
         "scale_pos_weight":1,
         "subsample":0.8,
         "min_child_weight":1,
         'tree_method': 'gpu_hist',
         'gpu_id': 0,}


cv_res= xgb.cv(param,dtrain,num_boost_round=2000,early_stopping_rounds=200,nfold=10,show_stdv=True)
print(cv_res)

      train-auc-mean  train-auc-std  test-auc-mean  test-auc-std
0           0.601932       0.016259       0.561471      0.026952
1           0.625592       0.017801       0.588932      0.033047
2           0.661744       0.015349       0.610015      0.033711
3           0.677567       0.015963       0.620970      0.032688
4           0.686260       0.015060       0.625058      0.033682
5           0.689116       0.015286       0.625732      0.033092
6           0.694082       0.013485       0.627303      0.032405
7           0.701104       0.011896       0.633905      0.030999
8           0.703292       0.014597       0.635537      0.030952
9           0.706941       0.013155       0.638558      0.032310
10          0.710380       0.010779       0.642031      0.031488
11          0.715892       0.009942       0.647206      0.031587
12          0.718502       0.009334       0.648509      0.030783
13          0.719674       0.008620       0.648802      0.031218
14          0.721199     

In [12]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)
x_perd = xgb.DMatrix(df_test[x_col])


In [13]:


dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)
x_perd = xgb.DMatrix(df_test[x_col])

evallist = [(dtrain,"train"),(dtest, 'eval')]
param = {
    'max_depth': 5,
    'eta': 0.008,
    'colsample_bytree': 0.5,
    'objective': 'binary:logistic',
    # 'verbosity':0,
    'scale_pos_weight': 1,
    'eval_metric': 'auc',
    'subsample': 0.55,
    'min_child_weight': 10,
    'gamma': 0.5,
    'seed': 666,
    'tree_method': 'gpu_hist',
    'gpu_id': 0,
}

bst = xgb.train(param, dtrain, 3000, evals=evallist,early_stopping_rounds=300)

[0]	train-auc:0.55028	eval-auc:0.54269
[1]	train-auc:0.55028	eval-auc:0.54269
[2]	train-auc:0.55186	eval-auc:0.54475
[3]	train-auc:0.55211	eval-auc:0.54453
[4]	train-auc:0.61576	eval-auc:0.57618
[5]	train-auc:0.61576	eval-auc:0.57618
[6]	train-auc:0.62532	eval-auc:0.57277
[7]	train-auc:0.64388	eval-auc:0.56716
[8]	train-auc:0.64905	eval-auc:0.57357
[9]	train-auc:0.64887	eval-auc:0.57352
[10]	train-auc:0.65010	eval-auc:0.57294
[11]	train-auc:0.65189	eval-auc:0.57587
[12]	train-auc:0.65431	eval-auc:0.57353
[13]	train-auc:0.65381	eval-auc:0.57282
[14]	train-auc:0.66027	eval-auc:0.58058
[15]	train-auc:0.66705	eval-auc:0.58495
[16]	train-auc:0.66852	eval-auc:0.58803
[17]	train-auc:0.66941	eval-auc:0.58743
[18]	train-auc:0.67827	eval-auc:0.59942
[19]	train-auc:0.67916	eval-auc:0.59881
[20]	train-auc:0.68024	eval-auc:0.59754
[21]	train-auc:0.68308	eval-auc:0.60038
[22]	train-auc:0.68998	eval-auc:0.59627
[23]	train-auc:0.68944	eval-auc:0.59564
[24]	train-auc:0.68928	eval-auc:0.59529
[25]	train

In [14]:
bst.predict(x_perd)

array([0.03440451, 0.00405227, 0.02419165, ..., 0.07202939, 0.16490541,
       0.00926685], dtype=float32)

In [15]:
xgb_pred_label = list(bst.predict(x_perd))

In [16]:
lgbcallback = [early_stopping(300), log_evaluation(10)]
param = {
        'task': 'train',
        "max_depth": 5,
        'boosting_type': 'gbdt',  # 设置提升类型
        'objective': 'binary',  # 目标函数
        'metric': 'auc',  # 评估函数
        'learning_rate': 0.01,  # 学习速率
        # 建树的特征选择比例
        'feature_fraction': 1,
        'bagging_fraction': 0.7,  # 建树的样本采样比例
        'bagging_freq': 1,  # k 意味着每 k 次迭代执行bagging
        'min_data_in_leaf': 1,
        'verbose': -1,  # <0 显示致命的, =0 显示错误 (警告), >0 显示信息
        'device_type': 'gpu',
        'seed':666,
    }

dtrain = lgb.Dataset(X_train, y_train)
dtest = lgb.Dataset(X_test, y_test)
bst1 = lgb.train(param, dtrain, 3000, valid_sets=[
            dtest], callbacks=lgbcallback)

Training until validation scores don't improve for 300 rounds
[10]	valid_0's auc: 0.621744
[20]	valid_0's auc: 0.63409
[30]	valid_0's auc: 0.639642
[40]	valid_0's auc: 0.645743
[50]	valid_0's auc: 0.647607
[60]	valid_0's auc: 0.659943
[70]	valid_0's auc: 0.659417
[80]	valid_0's auc: 0.664378
[90]	valid_0's auc: 0.670229
[100]	valid_0's auc: 0.668422
[110]	valid_0's auc: 0.669542
[120]	valid_0's auc: 0.673639
[130]	valid_0's auc: 0.674343
[140]	valid_0's auc: 0.677134
[150]	valid_0's auc: 0.682316
[160]	valid_0's auc: 0.684894
[170]	valid_0's auc: 0.687296
[180]	valid_0's auc: 0.688737
[190]	valid_0's auc: 0.690294
[200]	valid_0's auc: 0.691444
[210]	valid_0's auc: 0.692726
[220]	valid_0's auc: 0.693003
[230]	valid_0's auc: 0.691955
[240]	valid_0's auc: 0.693595
[250]	valid_0's auc: 0.695957
[260]	valid_0's auc: 0.699587
[270]	valid_0's auc: 0.701539
[280]	valid_0's auc: 0.70229
[290]	valid_0's auc: 0.704526
[300]	valid_0's auc: 0.708294
[310]	valid_0's auc: 0.708467
[320]	valid_0's auc

In [17]:
bst1.predict(df_test[x_col])

array([0.02663395, 0.00719871, 0.02901274, ..., 0.03579379, 0.15219753,
       0.02072502])

In [18]:
lgb_pred_label=list(bst1.predict(df_test[x_col]))

In [10]:
import numpy as np

from catboost import CatBoostClassifier, Pool

# initialize data


model = CatBoostClassifier(iterations=3000,
                           depth=3,
                           learning_rate=0.04,
                           loss_function='Logloss',
                           subsample=0.8,
                           early_stopping_rounds=300,
                           verbose=True)
# train the model
model.fit(X_train, y_train,eval_set=(X_test,y_test))
# make the prediction using the resulting model

cat_pred_label = model.predict_proba(df_test[x_col])[:,1]




C:\Users\tassadar667\AppData\Roaming\Python\Python39\site-packages\catboost\core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


0:	learn: 0.6325936	test: 0.6327756	best: 0.6327756 (0)	total: 153ms	remaining: 7m 38s
1:	learn: 0.5785373	test: 0.5787882	best: 0.5787882 (1)	total: 158ms	remaining: 3m 57s
2:	learn: 0.5304629	test: 0.5306416	best: 0.5306416 (2)	total: 163ms	remaining: 2m 42s
3:	learn: 0.4878209	test: 0.4881609	best: 0.4881609 (3)	total: 168ms	remaining: 2m 6s
4:	learn: 0.4504441	test: 0.4507886	best: 0.4507886 (4)	total: 173ms	remaining: 1m 43s
5:	learn: 0.4171628	test: 0.4175257	best: 0.4175257 (5)	total: 178ms	remaining: 1m 28s
6:	learn: 0.3875383	test: 0.3879724	best: 0.3879724 (6)	total: 183ms	remaining: 1m 18s
7:	learn: 0.3615188	test: 0.3619591	best: 0.3619591 (7)	total: 189ms	remaining: 1m 10s
8:	learn: 0.3385427	test: 0.3390163	best: 0.3390163 (8)	total: 194ms	remaining: 1m 4s
9:	learn: 0.3181405	test: 0.3185912	best: 0.3185912 (9)	total: 199ms	remaining: 59.6s
10:	learn: 0.3000420	test: 0.3004970	best: 0.3004970 (10)	total: 205ms	remaining: 55.7s
11:	learn: 0.2840619	test: 0.2845132	best: 0.

C:\Users\tassadar667\AppData\Roaming\Python\Python39\site-packages\catboost\core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


In [11]:
model.predict_proba(df_test[x_col])[:,1]

C:\Users\tassadar667\AppData\Roaming\Python\Python39\site-packages\catboost\core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


array([0.06506831, 0.01035321, 0.03154253, ..., 0.04264467, 0.18071922,
       0.01287547])

In [ ]:
roc_auc_score(y_test,model.predict_proba(X_test)[:,1])

0.7202397867359503

In [ ]:
pred_labels=[lgb_pred_label,xgb_pred_label]

In [ ]:
final_pred_label = np.exp(
    np.mean(np.log(pred_labels), axis=0))

In [ ]:
import time

date = time.strftime("%Y%m%d-%H%M%S", time.localtime())

res = pd.DataFrame({
    'cust_id': list(df_test["cust_id"]),
    'label': lgb_pred_label,
})
res.to_csv("./output/result"+date+".csv", encoding="utf-8", index=False)